In [9]:
import json
from pathlib import Path
from typing import Union

import pandas as pd
import requests
from geopy.geocoders import Nominatim  # for getting state that a Lat, Lon is in
from geopy.location import Location

from api_keys import OpenChargeMaps_API_KEY

## Get all US Charging Station Data

In [2]:
# see website for information:
# https://openchargemap.org/site/develop/api#/operations/get-poi
# compact: boolean - Set to true to remove reference data objects from output (just returns IDs for common reference data such as DataProvider etc).
# verbose: boolean - Set to false to get a smaller result set with null items removed.

url = "https://api.openchargemap.io/v3/poi"
querystring = {"maxresults": "200000", "countrycode": "US", "compact": "true", "verbose": "false", "key": OpenChargeMaps_API_KEY}
headers = {"Accept": "application/json"}
# All US data compact=true, verbose=false is >55,000 items for a size of ~78.5 MB

if Path("data/poi_data.json").exists():
    print("POI (station) data already exists, delete and rerun to recreate.")
else:
    response = requests.get(url, headers=headers, params=querystring)
    json_data = response.json()
    print("Total POI:", len(json_data))
    with open("data/poi_data.json", "w", encoding="utf8") as f:
        f.writelines(json.dumps(json_data, indent=4))
    print("Done writing file to 'data/poi_data.json'")

Total POI: 55195
Done writing file to 'data/poi_data.json'


## Get Reference Data

In [3]:
url_reference = "https://api.openchargemap.io/v3/referencedata"
querystring_reference = {"key": OpenChargeMaps_API_KEY}
headers_reference = {"Accept": "application/json"}

if Path("data/reference_data.json").exists():
    print("Reference data already exists, delete and rerun to recreate.")
else:
    response_reference = requests.get(url_reference, headers=headers_reference, params=querystring_reference)
    json_data_reference = response_reference.json()
    print("Total Reference Items:", len(json_data_reference))
    with open("data/reference_data.json", "w", encoding="utf8") as f:
        f.writelines(json.dumps(json_data_reference, indent=4))
    print("Done writing file to 'data/reference_data.json'")

Total Reference Items: 16
Done writing file to 'data/reference_data.json'


## Remove Unneeded fields from JSON
We want to reduce the file size of all the stations. This way the map and charts do not have to load as large of a file. This will save time on page loading.

In [2]:
# Read in JSON previously created from API request
json_data_path = Path("data/poi_data.json")

if json_data_path.exists():
    with json_data_path.open("r", encoding="utf8") as f:
        station_data = json.load(f)

In [3]:
KEYS_TO_KEEP = ["OperatorID", "UsageTypeID", "UsageCost", "NumberOfPoints", "DatePlanned", "StatusTypeID"]
ADDRESS_KEYS_TO_KEEP = ["Title", "AddressLine1", "AddressLine2", "Town", "StateOrProvince", "Postcode",
                        "Latitude", "Longitude"]
# CONNECTIONS_KEYS_TO_KEEP = ["ConnectionTypeID", "PowerKW", "CurrentTypeID"]  # Unused

In [4]:
def get_power_level_counts(power_kw_list: list[float]) -> tuple[int, int, int, int]:
    """Determine the number of Connections for each charging level 1, 2, 3.

    level_U is for unknown, 0.0, or other invalid data

    :param power_kw_list: list of floats representing power at each connection
    :return: count at each charging level (Unk, 1, 2, 3)
    """
    level_U = 0  # bad data, Unknown, missing data
    level_1 = 0
    level_2 = 0
    level_3 = 0
    for connection_power in power_kw_list:
        if connection_power <= 0:
            level_U += 1
        elif connection_power < 2.0:
            level_1 += 1
        elif connection_power < 40:
            level_2 += 1
        else:
            level_3 += 1
    return (level_U, level_1, level_2, level_3)

In [5]:
def fix_num_of_points_data(reduced_station_data: dict, actual_count: int, numOfPoints: int) -> dict:
    """Fix values in a station dictionary (`reduced_station_data`) when the count of
    charging stations is not equal to the number of points for that charing station.

    :param reduced_station_data: Station Data
    :param actual_count: Number of Connections
    :param numOfPoints: Number of Points from "NumberOfPoints" key
    :return: modified `reduced_station_data`
    """
    if numOfPoints == 0:
        # use actual count to replace "NumberOfPoints" value
        reduced_station_data["NumberOfPoints"] = actual_count
        return reduced_station_data
    elif actual_count == 1 and numOfPoints > actual_count:
        # if only one connection was given detail, assume all points are the same
        reduced_station_data["ConnectionTypeIDs"] = reduced_station_data["ConnectionTypeIDs"] * numOfPoints
        reduced_station_data["PowerKWs"] = reduced_station_data["PowerKWs"] * numOfPoints
        reduced_station_data["CurrentTypeIDs"] = reduced_station_data["CurrentTypeIDs"] * numOfPoints
        return reduced_station_data
    else:
        # assume actual connection count is correct and replace "NumberOfPoints" value
        reduced_station_data["NumberOfPoints"] = actual_count
        return reduced_station_data

In [6]:
# construct list of dicts from json that have removed unwanted fields
# doing this instead of flattening the json in pandas because there could
# be many fields in 'Connections' with different ConnectionTypeID and PowerKW values
good_station_data = []

# keep track of how many stations have "0" number of charging points
counter_0_numOfPoints = 0
# keep track of how many stations have an unknown Power Level (Level 0)
counter_unknown_power_level = 0

for station in station_data:
    # Immediately skip invalid stations
    if station["SubmissionStatusTypeID"] not in (200, 100):
        # station currently under review, rejected, or delisted = SKIP
        # there are apparently none in the retrieved data, but we will leave this in just in case
        continue

    reduced_station_data = {}
    # Extract Address, Location data from 'AddressInfo'
    for address_key, address_data in station["AddressInfo"].items():
        if address_key in ADDRESS_KEYS_TO_KEEP:
            reduced_station_data[address_key] = address_data

    connectionTypes = []  # type: list[int]
    powerKW = []          # type: list[float]
    currentTypes = []     # type: list[int]
    # extract data from 'Connections' list of dictionaries
    for single_connection in station["Connections"]:
        connection_type_id = single_connection.get("ConnectionTypeID", 0)   # 0=Unknown if null
        power_kw = single_connection.get("PowerKW", -1.0)                   # -1 if null
        current_type_id = single_connection.get("CurrentTypeID", 0)         # 0 if null, valid=[10, 20, 30]
        quantity = single_connection.get("Quantity", 1)  # default value in JSON is null

        # try to get a value for power from LevelID (1, 2, or 3) if PowerKW is missing or bad (-1, or 0.0)
        if power_kw <= 0:
            if level_id := single_connection.get("LevelID", None):
                # Determine a value use for the levels
                # https://www.transportation.gov/rural/ev/toolkit/ev-basics/charging-speeds
                if level_id == 1:       # Under 2kW
                    power_kw = 1.0
                elif level_id == 2:     # 2-40 kW
                    power_kw = 13.0
                elif level_id == 3:     # >= 40 kW
                    power_kw = 55.0
                else:  # Includes None
                    power_kw = -1.0     # still Unknown

        # create more values for connection lists if there is a value for quantity > 1
        if quantity > 1:
            connectionTypes.extend([connection_type_id] * quantity)
            powerKW.extend([power_kw] * quantity)
            currentTypes.extend([current_type_id] * quantity)
        else:
            connectionTypes.append(connection_type_id)
            powerKW.append(power_kw)
            currentTypes.append(current_type_id)
    
    reduced_station_data["ConnectionTypeIDs"] = connectionTypes
    reduced_station_data["PowerKWs"] = powerKW
    reduced_station_data["CurrentTypeIDs"] = currentTypes

    # add remaining data
    for station_key, station_value in station.items():
        if station_key in KEYS_TO_KEEP:
            reduced_station_data[station_key] = station_value
    
    # Connections (number of values in PowerKWs) and NumberOfPoints are not the same, try to fix
    if (s_actual := len(powerKW)) != (s_num := station.get('NumberOfPoints', 0)):
        counter_0_numOfPoints += 1
        # print(f"Station: {station['ID']} has conflicting number of connections: Actual={s_actual}, NumOfPoints={s_num}")
        # fix values
        reduced_station_data = fix_num_of_points_data(reduced_station_data, s_actual, s_num)

    # Determine number of charging connections at each power level (1, 2, or 3) for the station
    lvlUnk, lvl1, lvl2, lvl3 = get_power_level_counts(reduced_station_data["PowerKWs"])
    reduced_station_data["LUnknownCount"] = lvlUnk
    reduced_station_data["L1Count"] = lvl1
    reduced_station_data["L2Count"] = lvl2
    reduced_station_data["L3Count"] = lvl3
    # Warning Message for Unk
    if lvlUnk > 0:
        # uncomment to see list of stations that have 0.0 charging values after fix_num_of_points_data() has run
        # print(f"⚡ Station: {station['ID']} has {lvlUnk} 0.0 or less KW connections out of {reduced_station_data['NumberOfPoints']} connections.")
        counter_unknown_power_level += 1

    # add to new list
    good_station_data.append(reduced_station_data)

print("--+------+------+--")
print(f"Count of 0 NumberOfPoints charging stations fixed: {counter_0_numOfPoints}")
print(f"Number of Level 0 (No kW) Stations: {counter_unknown_power_level}")
print(f"Number of stations: {len(good_station_data)}")

--+------+------+--
Count of 0 NumberOfPoints charging stations fixed: 48996
Number of Level 0 (No kW) Stations: 637
Number of stations: 55195


In [22]:
# create DataFrame
station_df = pd.DataFrame(good_station_data)
station_df = station_df.astype({"OperatorID": "Int32",
                                "UsageTypeID": "Int32",
                                "StatusTypeID": "Int32",
                                "NumberOfPoints": "Int32",
                                "LUnknownCount": "Int32",
                                "L1Count": "Int32",
                                "L2Count": "Int32",
                                "L3Count": "Int32",})\
                        .replace("", None)  # There are some empty strings, remove them
# set default values for ID columns if they are null/NaN/None
station_df[["UsageTypeID", "StatusTypeID"]] = station_df[["UsageTypeID", "StatusTypeID"]].fillna(0)
station_df["OperatorID"] = station_df["OperatorID"].fillna(1)
station_df["UsageCost"] = station_df["UsageCost"].replace("FREE", "Free")

station_df.head()

,Title,AddressLine1,Town,StateOrProvince,Postcode,Latitude,Longitude,ConnectionTypeIDs,PowerKWs,CurrentTypeIDs,...,UsageTypeID,UsageCost,NumberOfPoints,StatusTypeID,LUnknownCount,L1Count,L2Count,L3Count,AddressLine2,DatePlanned
0,Walker Street,6 Walker Street,Lenox,Massachusetts,01242,42.356323,-73.285101,"[1, 1]","[6.3, 6.3]","[10, 10]",...,1,Free,2,50,0,0,2,0,NaN,NaN
1,Range USA,660 James L Hart Parkway,Ypsilanti Charter Township,Michigan,48197,42.226131,-83.622834,"[32, 2]","[50.0, 400.0]","[30, 30]",...,1,$2 + $0.35 kWh,2,50,0,0,0,2,NaN,NaN
2,Irving Oil / Circle K,206 Center Road,Fairfield,Maine,04937,44.594865,-69.617167,"[32, 32]","[120.0, 120.0]","[30, 30]",...,4,$0.45 per kWH,2,50,0,0,0,2,NaN,NaN
3,EVB N Lot,1201 Village Road,Dearborn,MI,48126,42.300885,-83.230529,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",...,6,Free,14,100,14,0,0,0,NaN,NaN
4,Carroll Shelby Way East,Carroll Shelby Way East,Dearborn,Michigan,48124,42.297993,-83.230880,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",...,6,Free,20,100,20,0,0,0,NaN,NaN


In [23]:
# Clean StateOrProvince values
# Only use short name (ie: 'TX'  NOT  'Texas' or 'texas')
print("Number of unique States:", len(station_df["StateOrProvince"].unique()))
print("Begin cleaning data...")

state_mapping = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "US Virgin Islands": "VI",
    # Other Variations
    "Washington State": "WA",
    "Washington DC": "DC",
    # Other Fixes
    "California - CA": "CA",
    "NEW  YORK": "NY",
    "MAS": "MA",
    "QC": "CA",  # 2 entries in Los Angeles
    # Counties
    "Chatham County": "GA",
    "San Luis Obispo County": "CA",
    "Unorganized Borough": "AK",
    "Fond du Lac County": "WI",
    "Quay County": "NM",
    "SACRAMENTO COUNTY": "CA",
    "SAN FRANCISCO CITY AND COUNTY": "CA",
    "LOS ANGELES": "CA",
    "LOS ANGELES COUNTY": "CA",
    "SAN DIEGO COUNTY": "CA",
    "NORTH CAROLINA / SURRY COUNTY": "NC",
    "CAPE GIRARDEAU COUNTY": "MO",
    "MECKLENBURG COUNTY": "NC",
    "Iron County": "MI",  # 1 entry in Michigan (there is more than 1 Iron County)
    # Spelling
    "SOUTH CAROLINE": "SC",
    "MINNESOTTA": "MN",
    "PENNSLYVANIA": "PA",
    "PENNSYLVANNIA": "PA",
    "LOUISANA": "LA",
    "TX\u200e": "TX",  # LeftToRight mark?????
    "NV\u200e": "NV",
    "CONNETICUT": "CT",
    "TENNESSE": "TN",
    "MISSISIIPPI": "MS",
    "VIRGINA": "VA",
    "LLLINOIS": "IL",
    "ILLNOIS": "IL",
    "ILLINOS": "IL",
    "INIDANA": "IN",
}
# Convert above dict to all upper case
cased_mapping_dict = {k.upper(): v for k,v in state_mapping.items()}
station_df["StateOrProvince"] = station_df["StateOrProvince"].str.replace(r"[,|\.]", "", regex=True)\
                                                             .str.upper()\
                                                             .str.strip()\
                                                             .replace(cased_mapping_dict)  # map will replace unfixed with NaN
# Drop Canada, India values
station_df = station_df[~station_df['StateOrProvince'].isin(["QUEBEC", "BRITISH COLUMBIA", "KA"])]

geolocator = Nominatim(user_agent="StateFinder")
valid_states = frozenset(('AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
                          'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD',
                          'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH',
                          'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
                          'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY',
                          "AS", "DC", "GU", "MP", "PR", "UM", "VI"))
def get_state_from_lat_long(row) -> Union[str, None]:
    state = row["StateOrProvince"]
    if state in valid_states:
        # No need to continue, state is already valid
        return state
    lat = row["Latitude"]
    lon = row["Longitude"]
    location = geolocator.reverse((lat, lon), exactly_one=True, addressdetails =True)  # type: Union[Location, None]
    if location:
        address = location.raw['address']
        state_full = address.get('state', '')
        country = address.get('country', '')
        if country != "United States":
            # Not in the US, need to get rid of it
            print(f"Location '{country}' not in US: ({lat}, {lon}), {row['Title']}: {row['AddressLine1']}, {row['Town']}")
            return None
        elif state_full == "":
            # No state name was given
            print(f"Location in US but has no State: ({lat}, {lon}), {row['Title']}: {row['AddressLine1']}, {row['Town']}")
            return None
        try:
            # State names from geolocator are full names, convert to short name
            return cased_mapping_dict[state_full.upper()]
        except KeyError:
            return None
    else:
        # geolocator could return None if location is invalid or some other issue
        print(f"Location not found: ({lat}, {lon}), {row['Title']}: {row['AddressLine1']}, {row['Town']}")
        return None

station_df["StateOrProvince"] = station_df.apply(get_state_from_lat_long, axis=1)
print("------")
print("Number of unique States (including None)", len(station_df["StateOrProvince"].unique()))
print(station_df["StateOrProvince"].unique())
print("------")
print("Number of invalid locations to drop", len(station_df.loc[station_df["StateOrProvince"].isna()]))
station_df = station_df.dropna(subset=["StateOrProvince"])
print("New DataFrame shape:", station_df.shape)

Number of unique States: 301
Begin cleaning data...
Location 'Canada' not in US: (43.65454, -79.3858), DoubleTree Downtown Toronto: 108 Chestnut St, Toronto
Location 'Canada' not in US: (45.32669, -75.8208), Best Western Plus Ottawa Kanata Hotel & Conference Centre: 1876 Robertson Road, Ottawa
Location 'مصر' not in US: (24.527271, 31.991007), SemaConnect Bangalore - 1: k r nagar  #123, Block 4, 50 Feet Road, Indira Nagar, Bengaluru
Location not found: (1.0, 1.0), Yucca Library: 5648 N 15th Ave, Phoenix
Location 'Canada' not in US: (48.53629, -80.4664), Tanny Convinience Store: 362 MacDougall Street, Matheson
Location 'Canada' not in US: (45.4294, -75.6848), Days Inn by Wyndham Ottawa: 319 Rideau Street, Ottawa
Location 'Canada' not in US: (43.81076, -79.2927), Excellence Auto Collision: 211 Silver Star Boulevard, Scarborough
Location 'Canada' not in US: (43.79137, -79.22747), AmbuEnergy Office - Private Use: 4-465 Milner Ave., Toronto
Location 'Canada' not in US: (43.7517, -79.68546), 

In [24]:
station_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55173 entries, 0 to 55194
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Title              55173 non-null  object 
 1   AddressLine1       55118 non-null  object 
 2   Town               55140 non-null  object 
 3   StateOrProvince    55173 non-null  object 
 4   Postcode           54970 non-null  object 
 5   Latitude           55173 non-null  float64
 6   Longitude          55173 non-null  float64
 7   ConnectionTypeIDs  55173 non-null  object 
 8   PowerKWs           55173 non-null  object 
 9   CurrentTypeIDs     55173 non-null  object 
 10  OperatorID         55173 non-null  Int32  
 11  UsageTypeID        55173 non-null  Int32  
 12  UsageCost          3678 non-null   object 
 13  NumberOfPoints     55173 non-null  Int32  
 14  StatusTypeID       55173 non-null  Int32  
 15  LUnknownCount      55173 non-null  Int32  
 16  L1Count            551

In [25]:
# export as CSV for SQL Database
station_df.to_csv("data/reduced_data.csv", encoding="utf8", header=True, index=False)

In [26]:
# export as JSON for web (map and plots)
station_df.to_json("data/reduced_data.json", orient="records", indent=2)

## Reference Data to CSVs and JSON

There are 5 fields in the main station data that use an ID value to reference more data.

They are:
- ConnectionTypeIDs
- CurrentTypeIDs
- OperatorID
- UsageTypeID
- StatusTypeID

We need to convert the json data from the reference data to CSV for use in a SQL database.

In [13]:
json_reference_data_path = Path("data/reference_data.json")
with json_reference_data_path.open("r", encoding="utf8") as f:
    reference_data = json.load(f)

In [14]:
# Connection Types (plugs/cables)
df_connection_types = pd.DataFrame(reference_data["ConnectionTypes"])
df_connection_types.sort_values("ID", inplace=True)
df_connection_types = df_connection_types[["ID", "Title", "FormalName", "IsDiscontinued", "IsObsolete"]]
print(df_connection_types.shape)
df_connection_types.head()

(42, 5)


,ID,Title,FormalName,IsDiscontinued,IsObsolete
39,0,Unknown,Not Specified,None,None
34,1,Type 1 (J1772),SAE J1772-2009,None,None
10,2,CHAdeMO,IEC 62196-3 Configuration AA,None,None
2,3,BS1363 3 Pin 13 Amp,BS1363 / Type G,None,None
1,4,Blue Commando (2P+E),None,None,None


In [15]:
# Current Type
df_current_types = pd.DataFrame(reference_data["CurrentTypes"])
# add a row for Unknown current type with ID=0
df_current_types = df_current_types.append({"Description": "Not Given","ID": 0,"Title": "Unknown"}, ignore_index=True) # type: ignore
df_current_types = df_current_types[["ID", "Title", "Description"]]
df_current_types

C:\Users\Zachary\AppData\Local\Temp\ipykernel_7652\2027009878.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_current_types = df_current_types.append({"Description": "Not Given","ID": 0,"Title": "Unknown"}, ignore_index=True) # type: ignore


,ID,Title,Description
0,10,AC (Single-Phase),Alternating Current - Single Phase
1,20,AC (Three-Phase),Alternating Current - Three Phase
2,30,DC,Direct Current
3,0,Unknown,Not Given


In [16]:
# Operator
df_operator = pd.DataFrame(reference_data["Operators"])
df_operator.drop(["WebsiteURL", "PhonePrimaryContact", "PhoneSecondaryContact", "AddressInfo",
                  "BookingURL", "ContactEmail", "FaultReportEmail", "IsRestrictedEdit"], axis=1, inplace=True)
df_operator.sort_values("ID", inplace=True)
df_operator = df_operator[["ID", "Title", "IsPrivateIndividual", "Comments"]]
print(df_operator.shape)
df_operator.head()

(667, 4)


,ID,Title,IsPrivateIndividual,Comments
2,1,(Unknown Operator),None,None
174,2,Elektrobay (UK),False,None
453,3,POD Point (UK),False,Part of Groupe EDF
83,5,ChargePoint,False,None
407,6,Nissan US Dealer Network,None,None


In [17]:
# Usage Type
df_usage = pd.DataFrame(reference_data["UsageTypes"])
df_usage.sort_values("ID", inplace=True)
df_usage = df_usage[["ID", "Title", "IsPayAtLocation", "IsMembershipRequired", "IsAccessKeyRequired"]]
print(df_usage.shape)
df_usage

(8, 5)


,ID,Title,IsPayAtLocation,IsMembershipRequired,IsAccessKeyRequired
0,0,(Unknown),None,None,None
4,1,Public,None,None,None
2,2,Private - Restricted Access,None,True,None
3,3,Privately Owned - Notice Required,None,None,None
5,4,Public - Membership Required,False,True,True
7,5,Public - Pay At Location,True,False,False
1,6,"Private - For Staff, Visitors or Customers",False,False,False
6,7,Public - Notice Required,False,False,False


In [18]:
# Status Type (active, ect)
df_status = pd.DataFrame(reference_data["StatusTypes"])
df_status = df_status[["ID", "Title", "IsOperational"]]
print(df_status.shape)
df_status

(10, 3)


,ID,Title,IsOperational
0,0,Unknown,None
1,10,Currently Available (Automated Status),True
2,20,Currently In Use (Automated Status),True
3,30,Temporarily Unavailable,True
4,50,Operational,True
5,75,Partly Operational (Mixed),True
6,100,Not Operational,False
7,150,Planned For Future Date,False
8,200,Removed (Decommissioned),False
9,210,Removed (Duplicate Listing),False


### Create CSVs and JSON

In [19]:
# CSVs
df_connection_types.to_csv("data/ref_connection_types.csv", encoding="utf8", header=True, index=False)
df_current_types.to_csv("data/ref_current_types.csv", encoding="utf8", header=True, index=False)
df_operator.to_csv("data/ref_operator_types.csv", encoding="utf8", header=True, index=False)
df_usage.to_csv("data/ref_usage_types.csv", encoding="utf8", header=True, index=False)
df_status.to_csv("data/ref_status_types.csv", encoding="utf8", header=True, index=False)

In [20]:
# JSON
combined_ref_json = {"ConnectionTypes": df_connection_types.to_dict(orient="records"),
                     "CurrentTypes": df_current_types.to_dict(orient="records"),
                     "OperatorTypes": df_operator.to_dict(orient="records"),
                     "UsageTypes": df_usage.to_dict(orient="records"),
                     "StatusTypes": df_status.to_dict(orient="records")}
with open("data/reduced_reference_data.json", "w") as f_out:
     json.dump(combined_ref_json, f_out, indent=2)

# The Final Data: Explained

## The `reduced_data.json` file
Contains data for all 55,000+ US Charging Stations with only the data we decided we needed.

Information from: [https://openchargemap.org/site/develop/api#/schemas/POI](https://openchargemap.org/site/develop/api#/schemas/POI)

Example Data for 1 station:
```json
{
    "Title":"",                 // [str]
    "AddressLine1":"",          // [str, null]
    "Town":"",                  // [str, null]  City Name
    "StateOrProvince":"",       // [str]        State Name (Two Letter)
    "Postcode":"",              // [str, null]  Zip Code
    "Latitude":40.1234567890,   // [float]
    "Longitude":-70.1234567890, // [float]
    "ConnectionTypeIDs":[       // Array[int]   ID for type of plug (Default = 0: Unknown)
      1,
      1
    ],
    "PowerKWs":[                // Array[float] ID for power in each connection, used to determine L1, L2, L3 Counts below
      6.3,
      6.3
    ],
    "CurrentTypeIDs":[          // Array[int]   ID for type of current [-1, 10, 20, 30]
      10,
      10
    ],
    "OperatorID":26,            // [int]        Who runs this charging station
    "UsageTypeID":1,            // [int]        ID for general restrictions on usage
    "UsageCost":"Free",         // [str, null]  Free text description of likely usage costs associated with this site. May include charging rate
    "NumberOfPoints":2,         // [int]        Number of charging points, should equal length of PowerKWs
    "StatusTypeID":50,          // [int]        ID for Available, Temp Unavailable, Removed, Future, ect.
    "LUnknownCount":0,          // [int]        Count of PowerKWs with value <= 0.0
    "L1Count":0,                // [int]        Count of Level 1 PowerKWs (< 2kW)
    "L2Count":2,                // [int]        Count of Level 2 PowerKWs (> 2 and < 40kW)
    "L3Count":0,                // [int]        Count of Level 3 PowerKWs (> 40kW) (Fast Charging)
    "AddressLine2":null,        // [str, null]  Optional extra address line
    "DatePlanned":null          // [DateTime string, null]  Planned operational date
},
```

The values of the different ID fields can be found in reference_reduced_data.json

## The `reduced_reference_data.json` file
This file contains the 5 Types that are represented in the main data as an ID.

Information from: [https://openchargemap.org/site/develop/api#/operations/get-referencedata](https://openchargemap.org/site/develop/api#/operations/get-referencedata)

A sample of the file is shown below.

```json
{
  "ConnectionTypes": [                // -> ConnectionTypeIDs
    {                                 //    type of connection/plug
      "ID": 0,                        // [int]  The ID number
      "Title": "Unknown",             // [str]  Short name of the connection
      "FormalName": "Not Specified",  // [str, null]  Long name of the connection
      "IsDiscontinued": null,         // [bool, null]
      "IsObsolete": null              // [bool, null]
    },
  ],
  "CurrentTypes": [                   // -> CurrentTypeIDs
    {                                 //    only 4 values: [0, 10, 20, 30]
      "ID": 0,                        // [int]
      "Title": "Unknown",             // [str]  Short name (AC (Single-Phase), AC (Three-Phase), DC)
      "Description": "Not Given"      // [str]  Long name
    }
  ],
  "OperatorTypes": [                  // -> OperatorID
    {                                 //    Who runs the station (Tesla, ChargePoint, eVgo)
      "ID": 1,                        // [int]
      "Title": "(Unknown Operator)",  // [str]        name of company
      "IsPrivateIndividual": null,    // [bool, null]
      "Comments": null                // [str, null]  extra detail
    }
  ],
  "UsageTypes": [                     // -> UsageTypeID
    {                                 //    8 values [0-7] (Public, Private, ect)
      "ID": 0,                        // [int]
      "Title": "(Unknown)",           // [str]
      "IsPayAtLocation": null,        // [bool, null]
      "IsMembershipRequired": null,   // [bool, null]
      "IsAccessKeyRequired": null     // [bool, null]
    }
  ],
  "StatusTypes": [                    // -> StatusTypeID
    {                                 //    10 values (Avaliable, Unavailable, Future, Removed, ect)
      "ID": 0,                        // [int]
      "Title": "Unknown",             // [str]
      "IsOperational": null           // [bool, null]
    }
  ]
}
```